In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

# 初期値

In [2]:
DAYS = 4000 # 生成日数
FROM_YEAR = 2010 #初期の年
FROM_MONTH = 3 #初期の月
EFFECTIVE_RATE = 0.3 # 有効率：取得する日数の割合
FIRST_STOCKS = 1000 # 在庫数の初期値

# データベースの作成

In [3]:
# 日付のリスト生成()
date_list = [datetime(FROM_YEAR, FROM_MONTH, 1) + timedelta(days=i) for i in range(DAYS)]
db = pd.DataFrame({"date": date_list})

# 入庫と出荷を追加
db["warehousing"] = [random.randint(0,20) for i in range(len(db))]
db["shipping"] = [random.randint(0,20) for i in range(len(db))]

# ランダムに日にちを絞り，並び替える
effective_list = random.sample([i for i in range(len(db))], int(EFFECTIVE_RATE * len(db))) 
db = db.iloc[effective_list, :]

db

,date,warehousing,shipping
3191,2018-11-25,20,3
2391,2016-09-16,11,14
2890,2018-01-28,6,6
2718,2017-08-09,17,0
2347,2016-08-03,17,5
...,...,...,...
370,2011-03-06,7,19
58,2010-04-28,14,7
3535,2019-11-04,14,5
3920,2020-11-23,6,17


# ロジック無しの場合

### 2020年4月分のみの処理

In [4]:
db1 = db[db["date"] >= datetime(2020, 4, 1)]
db2 = db1[db1["date"] <= datetime(2020, 4, 30)]
print(db2.sort_values("date"))

# 前月末残
stocks = FIRST_STOCKS
db1 = db[db["date"] >= datetime(2020, 4, 1)]
db1 = db1[db1["date"] <= datetime(2020, 4, 10)]
last_stocks = db1["warehousing"].sum() - db1["shipping"].sum() + stocks
#当月10日残
db1 = db[db["date"] >= datetime(2020, 4, 11)]
db1 = db1[db1["date"] <= datetime(2020, 4, 20)]
ten_stocks = db1["warehousing"].sum() - db1["shipping"].sum() + last_stocks
#当月20日残
db1 = db[db["date"] >= datetime(2020, 4, 21)]
db1 = db1[db1["date"] <= datetime(2020, 4, 30)]
twenty_stocks = db1["warehousing"].sum() - db1["shipping"].sum() + ten_stocks
#当月入庫
stocks_on_the_day = db2.warehousing.sum()

#積数
seki = last_stocks + ten_stocks + twenty_stocks + stocks_on_the_day

print("\n前月末残：{}\n当月10日残：{}\n当月20日残：{}\n当月入庫：{}".format(last_stocks, ten_stocks, twenty_stocks, stocks_on_the_day))
print("積数：{}".format(seki))

           date  warehousing  shipping
3688 2020-04-05           17         6
3689 2020-04-06           10        19
3691 2020-04-08           15         2
3692 2020-04-09           19        14
3700 2020-04-17           11         8
3702 2020-04-19           16        17
3711 2020-04-28            7        13

前月末残：1020
当月10日残：1022
当月20日残：1016
当月入庫：95
積数：3153


## 全体の処理

In [5]:
# 年月の範囲を修得する
db_sort = db.sort_values("date")
first_year = db_sort.iloc[0].date.year
first_month = db_sort.iloc[0].date.month
last_year = db_sort.iloc[-1].date.year
last_month = db_sort.iloc[-1].date.month
print(first_year, first_month, last_year, last_month)

# 範囲のリストを生成する．
year_list = list(range(first_year, last_year + 1))
month_list = list(range(1, 13)) * len(year_list)
month_list = month_list[first_month-1: -(12-last_month)]
year_list_all = []
if len(year_list) >= 2:
    year_list_all += [year_list[0]] * (12 - first_month + 1)
    for i in range(len(year_list) - 2):
        year_list_all += [year_list[i+1]] * 12
    year_list_all += [year_list[-1]] * last_month
elif len(year_list) == 1:
    year_list_all += [year_list[0]] * len(month_list)

print(year_list, month_list, len(month_list))
print(year_list_all, len(year_list_all))

2010 3 2021 2
[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021] [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2] 132
[2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016

In [6]:
print(db)

def make_stocks(year, month, stocks):
    if month == 12:
        next_year = year + 1
        next_month = 1
    else:
        next_year = year
        next_month = month + 1
    db1 = db[db["date"] >= datetime(year, month, 1)]
    db2 = db1[db1["date"] < datetime(next_year, next_month, 1)]
    #print(db2)
    
    # 前月末残
    db1 = db[db["date"] >= datetime(year, month, 1)]
    db1 = db1[db1["date"] <= datetime(year, month, 10)]
    last_stocks = db1["warehousing"].sum() - db1["shipping"].sum() + stocks
    #当月10日残
    db1 = db[db["date"] >= datetime(year, month, 11)]
    db1 = db1[db1["date"] <= datetime(year, month, 20)]
    ten_stocks = db1["warehousing"].sum() - db1["shipping"].sum() + last_stocks
    #当月20日残
    db1 = db[db["date"] >= datetime(year, month, 21)]
    db1 = db1[db1["date"] < datetime(next_year, next_month, 1)]
    twenty_stocks = db1["warehousing"].sum() - db1["shipping"].sum() + ten_stocks
    #当月入庫
    stocks_on_the_day = db2.warehousing.sum()
    
    return last_stocks, ten_stocks, twenty_stocks, stocks_on_the_day

# main
year = first_year
last_stocks_list = []
ten_stocks_list = []
twenty_stocks_list = []
stocks_on_the_day_list = []
twenty_stocks_m = FIRST_STOCKS
for month in month_list:
    last_stocks_m, ten_stocks_m, twenty_stocks_m, stocks_on_the_day_m = make_stocks(year, month, twenty_stocks_m)
    last_stocks_list.append(last_stocks_m)
    ten_stocks_list.append(ten_stocks_m)
    twenty_stocks_list.append(twenty_stocks_m)
    stocks_on_the_day_list.append(stocks_on_the_day_m)

    if month == 12:
        year += 1

storage_fee_dict = {
    "year": year_list_all,
    "month": month_list,
    "last_stocks": last_stocks_list,
    "ten_stocks": ten_stocks_list,
    "twenty_stocks": twenty_stocks_list,
    "stocks_on_the_day": stocks_on_the_day_list
}
storage_fee_db = pd.DataFrame(storage_fee_dict)
print(storage_fee_db)

           date  warehousing  shipping
3191 2018-11-25           20         3
2391 2016-09-16           11        14
2890 2018-01-28            6         6
2718 2017-08-09           17         0
2347 2016-08-03           17         5
...         ...          ...       ...
370  2011-03-06            7        19
58   2010-04-28           14         7
3535 2019-11-04           14         5
3920 2020-11-23            6        17
2922 2018-03-01            3         2

[1200 rows x 3 columns]
     year  month  last_stocks  ten_stocks  twenty_stocks  stocks_on_the_day
0    2010      3         1000         982           1000                 98
1    2010      4         1003        1030           1051                163
2    2010      5         1053        1043           1049                 98
3    2010      6         1069        1078           1064                114
4    2010      7         1058        1053           1046                 79
..    ...    ...          ...         ...          

In [7]:
# 積数
storage_fee_db["sekisu"] = storage_fee_db["last_stocks"] + storage_fee_db["ten_stocks"] + storage_fee_db["twenty_stocks"] + storage_fee_db["stocks_on_the_day"]
print("積数の合計：", storage_fee_db["sekisu"].sum())
storage_fee_db

積数の合計： 396231


,year,month,last_stocks,ten_stocks,twenty_stocks,stocks_on_the_day,sekisu
0,2010,3,1000,982,1000,98,3080
1,2010,4,1003,1030,1051,163,3247
2,2010,5,1053,1043,1049,98,3243
3,2010,6,1069,1078,1064,114,3325
4,2010,7,1058,1053,1046,79,3236
...,...,...,...,...,...,...,...
127,2020,10,885,887,883,70,2725
128,2020,11,873,881,867,72,2693
129,2020,12,854,854,818,62,2588
130,2021,1,788,799,794,70,2451


# その他

In [8]:
print(db.warehousing.sum(), db.shipping.sum(), db.warehousing.sum()-db.shipping.sum())

11768 11963 -195
